In [ ]:
import leo_satellites as sat
import csv
import ephem
import geometry_functions as geom
import math
from collections import defaultdict
import sys
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
def constellationFromSaVi(tcl_observed_date, tcl_file_name, satellite_per_orbit, number_of_obits, flat=False):
    constellation = list()
    with open(tcl_file_name, 'r') as tclfile:
        spaceX_SaVi = list(csv.reader(tclfile, delimiter='\n'))
        if len(spaceX_SaVi) != satellite_per_orbit * number_of_obits:
            raise Exception('The total number of satellites is not correct')

    for i in range(number_of_obits):
        orbit_i = list()
        for j in range(satellite_per_orbit):
            SaVi_line = spaceX_SaVi.pop(0)[0].split()
            for i in range (0, 2):
                SaVi_line.pop(0)
            to_add_sat = ephem.EarthSatellite()
            # TBD：历元是否影响结果-
            to_add_sat._epoch = tcl_observed_date #ok
            to_add_sat._n = geom.semi_major_to_mean_motion(float(SaVi_line[0])) #ok
            to_add_sat._e = float(SaVi_line[1]) + math.pow(10, -30) #ok
            to_add_sat._inc = float(SaVi_line[2]) #ok
            to_add_sat._raan = float(SaVi_line[3]) #ok
            to_add_sat._ap = float(SaVi_line[4]) #ok
            to_add_sat._M = geom.time_to_periapsis_to_mean_anomaly(float(SaVi_line[5]),to_add_sat._n) #ok
            to_add_sat._drag = 0
            orbit_i.append(to_add_sat)
        if flat is True:
            constellation.extend(orbit_i)
        else:
            constellation.append(orbit_i)
    return constellation

def get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n):
    observer_list = []
    
    for i in range(lat_n):
        for j in range(lon_n):
            lat = (lat_end - lat_start) / lat_n * i + lat_start
            lon = (lon_end - lon_start) / lon_n * j + lon_start
            observer = ephem.Observer()
            observer.lat = ephem.degrees(lat)
            observer.lon = ephem.degrees(lon)
            observer.date = observe_time
            observer_list.append(observer)
    return observer_list

In [ ]:
def get_max_accessible_seconds_since(accessible_time_set, t):
    if t not in accessible_time_set:
        return 0
    max_t = t + 1
    max_accessible_seconds = 0
    while True:
        if max_t in accessible_time_set:
            max_accessible_seconds += 1
        else:
            return max_accessible_seconds
        max_t += 1

In [ ]:
def get_max_accessible_seconds(
    t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    return get_max_accessible_seconds_since(accessible_time[observer_idx][satellite_idx], t)

def get_rss(t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    satellite = satellite_list[satellite_idx]
    observer = observer_list[observer_idx]
    satellite.compute(observer)
    distance_in_meter = satellite.range
    rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
    return rss

def get_free_channel(t, max_channel_per_satellite, satellite_idx, satellite_list, observer_idx, observer_list, accessible_time, satellite_link_count):
    return max_channel_per_satellite - satellite_link_count[satellite_idx]

In [ ]:
def do_simulate(
    observe_start_time,
    observe_total_seconds,
    satellite_list,
    lat_start,
    lat_end,
    lon_start,
    lon_end,
    lat_n,
    lon_n,
    min_service_degrees,
    max_link_per_satellite,
    no_signal_rss=NO_SIGNAL_RSS,
    name_and_evaluate_func_info=[("by_mvt", get_max_accessible_seconds)],
    ):
    accessible_time = defaultdict(lambda: defaultdict(set))
    for t in range(observe_total_seconds):
        observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
        observer_list = get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n)
        for i, observer in enumerate(observer_list):
            for j, satellite in enumerate(satellite_list):
                satellite.compute(observer)
                if satellite.alt >= min_service_degrees:
                    accessible_time[i][j].add(t)

    observer_rss_log = defaultdict(lambda: defaultdict(list))
    observer_handover_log = defaultdict(lambda: defaultdict(list))
    observer_handover_detail_log = defaultdict(lambda: defaultdict(list))

    for name, evaluate_func in name_and_evaluate_func_info:
        satellite_link_count = defaultdict(int)
        observer2linked_statellite = [None] * len(observer_list)

        for t in range(observe_total_seconds):
            observe_time = ephem.date(observe_start_time + t / 24 / 60 / 60)
            observer_list = get_observer_list(observe_time, lat_start, lat_end, lon_start, lon_end, lat_n, lon_n)
            for i, observer in enumerate(observer_list):
                previous_satellite_idx = observer2linked_statellite[i]
                # if the observer is not connected, or the previous connected satellite is out of service
                if previous_satellite_idx is None or t not in accessible_time[i][previous_satellite_idx]:
                    # try to find an available satellite
                    candidate_list = []
                    for satellite_idx in accessible_time[i]:
                        if t in accessible_time[i][satellite_idx]:
                            if satellite_link_count[satellite_idx] < max_link_per_satellite:
                                candidate_list.append(satellite_idx)
                    if len(candidate_list) > 0:
                        # we can find a candidate
                        best_satellite_idx = max(candidate_list, key=lambda satellite_idx: evaluate_func(
                            t, 
                            max_link_per_satellite,
                            satellite_idx,
                            satellite_list,
                            i,
                            observer_list,
                            accessible_time,
                            satellite_link_count
                        ))
                        if previous_satellite_idx is not None:
                            satellite_link_count[previous_satellite_idx] -= 1
                            observer_handover_log[name][i].append(t)
                        observer2linked_statellite[i] = best_satellite_idx
                        satellite_link_count[best_satellite_idx] += 1
                        satellite = satellite_list[best_satellite_idx]
                        satellite.compute(observer)
                        distance_in_meter = satellite.range
                        rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
                        observer_rss_log[name][i].append(rss)
                        observer_handover_detail_log[name][i].append(1)
                    else:
                        # we cannot find a candidate, no handover happens, but there's no signal at all
                        observer_rss_log[name][i].append(no_signal_rss)
                        observer_handover_detail_log[name][i].append(0)
                else:
                    # observer is ok, we records its signal 
                    linked_satellite_idx = observer2linked_statellite[i]
                    satellite = satellite_list[linked_satellite_idx]
                    satellite.compute(observer)
                    distance_in_meter = satellite.range
                    rss = -43 - 40 * math.log(distance_in_meter / 1000 / 1000, 10)
                    observer_rss_log[name][i].append(rss)
                    observer_handover_detail_log[name][i].append(0)
    return {
        'observer_rss_log': observer_rss_log,
        'observer_handover_log': observer_handover_log,
        'observer_handover_detail_log': observer_handover_detail_log,
    }

In [ ]:
SATELLITES_PER_ORBIT = 22
NUMBER_OF_ORBITS = 72
TCL_FILE_NAME = './spacex_constellation/spacex_phase1_550km.tcl'

LAT_START = ephem.degrees('30:40:00')
LAT_END = ephem.degrees('31:53:00')
LON_START = ephem.degrees('120:51:00')
LON_END = ephem.degrees('122:12:00')

MIN_SERVICE_DEGREES = ephem.degrees('40')

TCL_OBSERVED_DATE = ephem.date('2022/09/21 00:00:00')
MAX_LINK_PER_SATELLITE = 10

NO_SIGNAL_RSS = -100

## don't consider max_link_per_satellite

### single satellite v.s. different evalute func

In [ ]:
satellite_list = constellationFromSaVi(TCL_OBSERVED_DATE, TCL_FILE_NAME, SATELLITES_PER_ORBIT, NUMBER_OF_ORBITS, flat=True)
ret = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    LAT_START,
    LAT_END,
    LON_START,
    LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
        ("by_ms", get_rss),
    ]
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret['observer_rss_log']['by_mvt'][0],
                    mode='lines',
                    name='by_mvt'))
 
fig.update_layout(
    title="single observer rss v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret['observer_rss_log']['by_ms'][0],
                    mode='lines',
                    name='by_ms', line_color='red'))
 
fig.update_layout(
    title="single observer rss v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret['observer_rss_log']['by_mvt'][0],
                    mode='lines',
                    name='by_mvt'))
fig.add_trace(go.Scatter(y=ret['observer_rss_log']['by_ms'][0],
                    mode='lines',
                    name='by_ms'))
 
fig.update_layout(
    title="single observer rss v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

### single satellite v.s. different min degree


In [ ]:
satellite_list = constellationFromSaVi(TCL_OBSERVED_DATE, TCL_FILE_NAME, SATELLITES_PER_ORBIT, NUMBER_OF_ORBITS, flat=True)
ret_40 = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    LAT_START,
    LAT_END,
    LON_START,
    LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
        ("by_ms", get_rss),
    ]
)
ret_20 = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    LAT_START,
    LAT_END,
    LON_START,
    LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('20'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
        ("by_ms", get_rss),
    ]
)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret_40['observer_rss_log']['by_mvt'][0],
                    mode='lines',
                    name='min alt = 40'))
fig.add_trace(go.Scatter(y=ret_20['observer_rss_log']['by_mvt'][0],
                    mode='lines',
                    name='min alt = 20'))
 
fig.update_layout(
    title="single observer rss v.s. time, by mvt",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ret_40['observer_rss_log']['by_ms'][0],
                    mode='lines',
                    name='min alt = 40'))
fig.add_trace(go.Scatter(y=ret_20['observer_rss_log']['by_ms'][0],
                    mode='lines',
                    name='min alt = 20'))
 
fig.update_layout(
    title="single observer rss v.s. time by ms",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
ret = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    1600,
    satellite_list,
    LAT_START,
    LAT_END,
    LON_START,
    LON_END,
    1,
    1,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
        ("by_ms", get_rss),
    ]
)

fig = go.Figure()
fig.add_trace(go.Scatter(y=np.cumsum(ret['observer_handover_detail_log']['by_mvt'][0]),
                    mode='lines',
                    name='by mvt'))
fig.add_trace(go.Scatter(y=np.cumsum(ret['observer_handover_detail_log']['by_ms'][0]),
                    mode='lines',
                    name='by ms'))

fig.update_layout(
    title="handover count v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
by_mvt_handover_cnt = []
by_ms_handover_cnt = []
min_degree_list = list(range(10, 40, 1))
for min_degree in min_degree_list:
    print("process....", min_degree)
    ret = do_simulate(
        ephem.date('2022/09/21 00:00:00'),
        1600,
        satellite_list,
        LAT_START,
        LAT_END,
        LON_START,
        LON_END,
        1,
        1,
        min_service_degrees=ephem.degrees(str(min_degree)),
        max_link_per_satellite=10,
        name_and_evaluate_func_info=[
            ("by_mvt", get_max_accessible_seconds),
            ("by_ms", get_rss),
        ]
    )
    by_mvt_handover_cnt.append(sum(ret['observer_handover_detail_log']['by_mvt'][0]))
    by_ms_handover_cnt.append(sum(ret['observer_handover_detail_log']['by_ms'][0]))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=min_degree_list, y=by_mvt_handover_cnt,
                    mode='lines',
                    name='mvt'))
fig.add_trace(go.Scatter(x=min_degree_list, y=by_ms_handover_cnt,
                    mode='lines',
                    name='ms'))

fig.update_layout(
    title="handover count v.s. minimum degree",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

## Consider Link limit

In [ ]:
simulation_time = 1600
satellite_list = constellationFromSaVi(TCL_OBSERVED_DATE, TCL_FILE_NAME, SATELLITES_PER_ORBIT, NUMBER_OF_ORBITS, flat=True)
ret = do_simulate(
    ephem.date('2022/09/21 00:00:00'),
    simulation_time,
    satellite_list,
    LAT_START,
    LAT_END,
    LON_START,
    LON_END,
    5,
    5,
    min_service_degrees=ephem.degrees('40'),
    max_link_per_satellite=10,
    name_and_evaluate_func_info=[
        ("by_mvt", get_max_accessible_seconds),
        ("by_ms", get_rss),
        ("by_mfc", get_free_channel),
    ]
)

In [ ]:
observer_rss_log = ret['observer_rss_log']
block_count_across_time = defaultdict(list)
has_signal_count_across_time = defaultdict(list)
average_rss_across_time = defaultdict(list)
for name in observer_rss_log:
    for t in range(simulation_time):
        has_signal_count = sum([1 for i in observer_rss_log[name] if observer_rss_log[name][i][t] != NO_SIGNAL_RSS])
        has_signal_count_across_time[name].append(has_signal_count)
        block_count =  sum([1 for i in observer_rss_log[name] if observer_rss_log[name][i][t] == NO_SIGNAL_RSS])
        block_count_across_time[name].append(block_count)
        average_rss = sum([observer_rss_log[name][i][t] for i in observer_rss_log[name] if observer_rss_log[name][i][t] != NO_SIGNAL_RSS]) / has_signal_count
        average_rss_across_time[name].append(average_rss)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=np.cumsum(block_count_across_time['by_mvt']),
                    mode='lines',
                    name='by mvt'))

fig.add_trace(go.Scatter(y=np.cumsum(block_count_across_time['by_ms']),
                    mode='lines',
                    name='by ms'))
fig.add_trace(go.Scatter(y=np.cumsum(block_count_across_time['by_mfc']),
                    mode='lines',
                    name='by mfc'))

fig.update_layout(
    title="block count v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=has_signal_count_across_time['by_mvt'],
                    mode='lines',
                    name='by mvt'))

fig.add_trace(go.Scatter(y=has_signal_count_across_time['by_ms'],
                    mode='lines',
                    name='by ms'))
fig.add_trace(go.Scatter(y=has_signal_count_across_time['by_mfc'],
                    mode='lines',
                    name='by mfc'))

fig.update_layout(
    title="observer with rss count v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=average_rss_across_time['by_mvt'],
                    mode='lines',
                    name='by mvt'))

fig.add_trace(go.Scatter(y=average_rss_across_time['by_ms'],
                    mode='lines',
                    name='by ms'))
fig.add_trace(go.Scatter(y=average_rss_across_time['by_mfc'],
                    mode='lines',
                    name='by mfc'))

fig.update_layout(
    title="mean rss v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()

In [ ]:
observer_handover_detail_log = ret['observer_handover_detail_log']

handover_log_across_time = defaultdict(list)
for name in observer_rss_log:
    for t in range(simulation_time):
        handover_count_at_time_t = sum([observer_handover_detail_log[name][i][t] for i in observer_handover_detail_log[name]])
        handover_log_across_time[name].append(handover_count_at_time_t)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=np.cumsum(handover_log_across_time['by_mvt']),
                    mode='lines',
                    name='by mvt'))
fig.add_trace(go.Scatter(y=np.cumsum(handover_log_across_time['by_ms']),
                    mode='lines',
                    name='by ms'))
fig.add_trace(go.Scatter(y=np.cumsum(handover_log_across_time['by_mfc']),
                    mode='lines',
                    name='by mfc'))

fig.update_layout(
    title="handover count v.s. time",
    width=900,
    height=400,
    showlegend=True,
)
fig.show()